# SDGSat - Find Matching TIF-s

In [16]:
import os
import rasterio
import geopandas as gpd
from pathlib import Path
from dotenv import load_dotenv
from shapely.geometry import box

from affine import Affine
from rasterio.crs import CRS
from rasterio.coords import BoundingBox
from shapely.geometry.polygon import Polygon
from geopandas.geodataframe import GeoDataFrame

In [2]:
_ = load_dotenv()

## Create a Shape File from GeoPandas DataFrame

https://doc.arcgis.com/en/arcgis-online/reference/shapefiles.htm#:~:text=A%20shapefile%20is%20an%20Esri,add%20it%20to%20the%20map.

https://www.ibm.com/docs/en/ias?topic=formats-shapefile-representation

In [8]:
# Nigeria, Lagos, 6.52°N, 3.37°E, 2024-12-23
tif_path = "../data/sdgsat/KX10_GIU_20241223_E3.08_N7.67_202400138080_L4A/KX10_GIU_20241223_E3.08_N7.67_202400138080_L4A_A_LH.tif"

with rasterio.open(tif_path) as src:
    bounds: BoundingBox = src.bounds  # left, bottom, right, top
    crs: CRS = src.crs
    transform: Affine = src.transform

bbox: Polygon = box(*bounds)
gdf = gpd.GeoDataFrame({"geometry": [bbox]}, crs=crs)

# re-project to WGS84 (lat/lon), required by blackmarblepy
gdf = gdf.to_crs("EPSG:4326")

gdf.to_file("shapes/lagos/lagos.shp")

## Finding Matching TIF-s

In [ ]:
gdf: GeoDataFrame = gpd.read_file("shapes/lagos/lagos.shp")

data_path = Path(os.path.abspath('')).parent / "data"
filepaths = list(data_path.rglob('*_LH.tif'))

matching_files = []

for filepath in filepaths:
    with rasterio.open(filepath) as src:
        bounds = src.bounds
        raster_poly = box(bounds.left, bounds.bottom, bounds.right, bounds.top)
        if gdf.to_crs(src.crs).intersects(raster_poly).any():
            matching_files.append(filepath)

In [15]:
matching_files

[PosixPath('/Users/jan.kokla/Documents/EPFL/conflict-monitoring-ntl/data/sdgsat/KX10_GIU_20241223_E3.08_N7.67_202400138080_L4A/KX10_GIU_20241223_E3.08_N7.67_202400138080_L4A_A_LH.tif'),
 PosixPath('/Users/jan.kokla/Documents/EPFL/conflict-monitoring-ntl/data/sdgsat/KX10_GIU_20241223_E3.08_N7.67_202400138080_L4A/KX10_GIU_20241223_E3.08_N7.67_202400138080_L4A_B_LH.tif')]